In [7]:
!pip install torch chess transformers datasets peft

  Using cached peft-0.18.0-py3-none-any.whl.metadata (14 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
Using cached peft-0.18.0-py3-none-any.whl (556 kB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)


In [8]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\Users\Harikrishna\anaconda3\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [1]:
MODEL_NAME = "diabolic6045/gemma-2-2b-chess"

# Load dataset
dataset = load_dataset(
    "text",
    data_files={"train": "data/anand_train.txt"}
)

NameError: name 'load_dataset' is not defined

In [ ]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )

tokenized_ds = dataset.map(tokenize, batched=True)

In [ ]:
# Load model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

In [ ]:
# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Training args
training_args = TrainingArguments(
    output_dir="anand_gemma_lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=500,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"]
)

trainer.train()

In [ ]:
# Save model
model.save_pretrained("anand_gemma_lora")
tokenizer.save_pretrained("anand_gemma_lora")